<a href="https://colab.research.google.com/github/kartikbatra056/AllAboutTimeSeries/blob/main/TimeSeriesWithLSTMandDCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Dataset from kaggle

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d 'rohanrao/nifty50-stock-market-data'

 50% 9.00M/18.0M [00:00<00:00, 35.1MB/s]
100% 18.0M/18.0M [00:00<00:00, 51.8MB/s]


In [5]:
!unzip '/content/nifty50-stock-market-data.zip'

Archive:  /content/nifty50-stock-market-data.zip
  inflating: ADANIPORTS.csv          
  inflating: ASIANPAINT.csv          
  inflating: AXISBANK.csv            
  inflating: BAJAJ-AUTO.csv          
  inflating: BAJAJFINSV.csv          
  inflating: BAJFINANCE.csv          
  inflating: BHARTIARTL.csv          
  inflating: BPCL.csv                
  inflating: BRITANNIA.csv           
  inflating: CIPLA.csv               
  inflating: COALINDIA.csv           
  inflating: DRREDDY.csv             
  inflating: EICHERMOT.csv           
  inflating: GAIL.csv                
  inflating: GRASIM.csv              
  inflating: HCLTECH.csv             
  inflating: HDFC.csv                
  inflating: HDFCBANK.csv            
  inflating: HEROMOTOCO.csv          
  inflating: HINDALCO.csv            
  inflating: HINDUNILVR.csv          
  inflating: ICICIBANK.csv           
  inflating: INDUSINDBK.csv          
  inflating: INFRATEL.csv            
  inflating: INFY.csv                
 

In [84]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
plt.style.use('ggplot')
%matplotlib inline

In [85]:
df=pd.read_csv('/content/RELIANCE.csv',parse_dates=['Date'])
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2000-01-03,RELIANCE,EQ,233.05,237.50,251.70,237.50,251.70,251.70,249.37,4456424,1.111319e+14,NaN,NaN,NaN
1,2000-01-04,RELIANCE,EQ,251.70,258.40,271.85,251.30,271.85,271.85,263.52,9487878,2.500222e+14,NaN,NaN,NaN
2,2000-01-05,RELIANCE,EQ,271.85,256.65,287.90,256.65,286.75,282.50,274.79,26833684,7.373697e+14,NaN,NaN,NaN
3,2000-01-06,RELIANCE,EQ,282.50,289.00,300.70,289.00,293.50,294.35,295.45,15682286,4.633254e+14,NaN,NaN,NaN
4,2000-01-07,RELIANCE,EQ,294.35,295.00,317.90,293.00,314.50,314.55,308.91,19870977,6.138388e+14,NaN,NaN,NaN


In [87]:
def train_test_data(data):
     '''
     Scale and split dataset.
     parameters:
       data:Time series dataset 
     returns: 
       train_data: Training data
       valid_data: Validation Data
     '''
     
     scaler=MinMaxScaler()
     
     data=data.asfreq('D')

     data.fillna(method='bfill',inplace=True)

     data.fillna(method='ffill',inplace=True)

     train_data=pd.DataFrame(index=data.iloc[:-int(len(data)*0.2)].index)

     valid_data=pd.DataFrame(index=data.iloc[-int(len(data)*0.2):].index)
     
     train_data['VWAP']=scaler.fit_transform(np.reshape(data.iloc[:-int(len(data)*0.2)].values,(-1,1))).squeeze(1)
     
     valid_data['VWAP']=scaler.transform(np.reshape(data.iloc[-int(len(data)*0.2):].values,(-1,1))).squeeze(1)
     
     return train_data,valid_data

In [92]:
class RelianceDataset(Dataset):
      def __init__(self,data,T=10,step=1):
          super().__init__()
          self.data=np.reshape(data.values,(data.shape[0],1))
          self.T=T
          self.chunks=torch.FloatTensor(self.data).unfold(0,self.T,step).permute(0,2,1)
          
      def __getitem__(self,index:int): 
          
           x=self.chunks[index,:-1,0]
           y=self.chunks[index,-1,0]   
           return x,y
      
      def __len__(self):
          
          return self.chunks.size(0)

In [94]:
df.set_index('Date',inplace=True)
train_data,valid_data=train_test_data(df['VWAP'])
train_data.shape,valid_data.shape

((6086, 1), (1521, 1))

In [95]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [96]:
class LSTMModel(nn.Module):
    def __init__(self,f_dim,h_dim,n_layers=1,drop_l=0.05,bidirect=False):
        
        super(LSTMModel,self).__init__() 
        
        if bidirect: 
          self.n_layers=2*n_layers
        else:
          self.n_layers=n_layers  

        self.h_dim=h_dim

        self.lstm=nn.LSTM(f_dim,h_dim,n_layers,batch_first=True,dropout=drop_l,bidirectional=bidirect)

        self.fc=nn.Linear(2*h_dim,out_dim)

    def forward(self,Input):     
    
        h0,c0=torch.zeros((self.n_layers,Input.size(0),self.h_dim)).to(device),torch.zeros((self.n_layers,Input.size(0),self.h_dim)).to(device)

        output,(h0,c0)=self.lstm(Input,(h0,c0))

        out=sel.fc(output[-1])

        return out 